# Import Packages

In [2]:
# Other
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
# Local
from networks import XCEPTION
from helper import load_dataset
from helper import split_data
from helper import preprocess_input

Using TensorFlow backend.


# Setup Training Paramters

In [3]:
total_epochs = 400 # num of iterations through network
input_shape = (64, 64, 1) # shape of image data
net_patience = 50 # training stopped after 50 epochs if no improvement
verbose_lvl = 2 # level of information detail output from tasks
validation_split = .2 # 
batch_num = 32 # number of images sent through network for training per gradient update
num_classes = 7 # number of possible classification outputs

# Keras Data Generator

In [4]:
# configures image data preparation and augmentation
data_generator = ImageDataGenerator(
    featurewise_center=False, # irrelevant, standard param
    featurewise_std_normalization=False, # irrelevant, standard param
    rotation_range=10, # randomnly rotates an image by 10 degrees
    width_shift_range=0.1, # randomnly shifts width dimensions by .1 of a pixel
    height_shift_range=0.1, # randomnly shifts height dimensions by .1 of a pixel
    zoom_range=.1, # randomnly zooms images by .1 of a pixel
    horizontal_flip=True # randomnly flips an image horizontally
)

# Compile Model

In [5]:
# initialise model with expected image input shape and number of possible output classes
model = XCEPTION(input_shape, num_classes)

# Adam Optimizer - Cat Cross Entropy (Loss Calc) - Accuracy Metrics (Loss Value)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary() # print a model summary

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 62, 62, 8)    72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 62, 62, 8)    32          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 62, 62, 8)    0           batch_normalization_1[0][0]      
_____________________________________

# Setup Keras Callbacks/Logging

In [6]:
log_file_path = 'models/emotion/training.log' # set path of logging file for training
csv_logger = CSVLogger(log_file_path, append=False) # setup Keras CSVLogger using file path

# stop training if patience is met
early_stopping = EarlyStopping('val_loss', patience=net_patience)

# reduce the Learning Rate (LR) when network is not improving
reduce_LR = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(net_patience/4), verbose=1)

trained_model_path = 'models/emotion/XCEPTION' # set path for trained model file

# the saved models will show the epoch number and accuracy based on validation set
model_name = trained_model_path + '_{epoch:02d}-{val_acc:.2f}.hdf5'

# save the latest best model only, not all models
model_checkpoint = ModelCheckpoint(model_name, 'val_loss', verbose=1, save_best_only=True)

callbacks = [model_checkpoint, csv_logger, early_stopping, reduce_LR]

# Load Data & Train Model

In [ ]:
# load the dataset faces and emotions
faces, emotions = load_dataset()

# ensure each faces pixels values are between 0 & 255 (grayscale)
faces = preprocess_input(faces)

# ensure number of possible classification outputs is correct
num_classes = emotions.shape

# split data into training and validation data (80% training, 20% validation)
training_data, validation_data = split_data(faces, emotions, validation_split)

# assign training faces and corresponding training emotions
train_faces, train_emotions = training_data

In [ ]:
# work on training the model batch by batch using the data generator
# flow() will accept the training data and labels according to batch size
model.fit_generator(
    data_generator.flow(
        train_faces, train_emotions, batch_num),
        steps_per_epoch = len(train_faces)/batch_num,
        epochs = total_epochs,
        verbose = 1,
        callbacks = callbacks,
        validation_data = validation_data
)

In [1]:
val_faces, val_emotions = validation_data

result = model.evaluate(val_faces, val_emotions, verbose=0)

NameError: name 'validation_data' is not defined